
Graph filtration to Cuntz--Krieger K-theory profile over a finite field.

This example demonstrates a stepwise profile computed from a proximity graph filtration.

# Pipeline

1. Sample a finite point cloud $X \subset \mathbb R^2$.
2. Build a proximity graph filtration $(G_s)_{s=0}^{T-1}$ on the fixed vertex set
   $V = \{0,\dots,n-1\}$ with nondecreasing thresholds
   $t_0 \le \cdots \le t_{T-1}$.
3. For each step $s$, form the adjacency matrix $A_s$ of $G_s$.
4. Compute finite field dimensions associated to the Cuntz--Krieger presentation matrix
   $I - A_s^{\mathsf T}$ and record them as arrays.
5. Plot the resulting functions against the thresholds.

# Mathematical meaning

Let $A \in \{0,1\}^{n\times n}$ be an adjacency matrix. Consider the linear map over
$\mathbb F_p$

\begin{align}\phi_A = I - A^{\mathsf T} : \mathbb F_p^n \to \mathbb F_p^n.\end{align}

The routine :func:`homolipop.kgraph.k_theory_cuntz_krieger_Fp` is assumed to compute

\begin{align}\dim_{\mathbb F_p} \operatorname{coker}(\phi_A),
   \qquad
   \dim_{\mathbb F_p} \ker(\phi_A).\end{align}

These are finite field linear invariants attached to $A$. Under standard hypotheses on
a 0--1 matrix $A$, the integral $K$-groups of the Cuntz--Krieger algebra
$\mathcal O_A$ satisfy algebraic descriptions in terms of the abelian groups
$\operatorname{coker}(I - A^{\mathsf T})$ and $\ker(I - A^{\mathsf T})$.
This script does not compute persistent $K$-theory. It computes the finite field
dimensions of these presentation groups at each filtration step.

# Reproducibility

All randomness in this file is controlled by NumPy's ``default_rng`` with a fixed seed.
The filtration is deterministic given the point cloud and filtration parameters.

# Implementation notes

The proximity filtration is constructed without self loops, and adjacency matrices are
materialized on demand with integer dtype ``np.int8``.


In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np

from homolipop.graph_filtration import proximity_graph_filtration
from homolipop.kgraph import k_theory_cuntz_krieger_Fp
from homolipop.kplotting import KTheoryProfile, plot_k_theory_profile


def main() -> None:
    """
    Compute and plot a finite field K-theory profile along a graph filtration.

    The script samples ``60`` points in the unit square, builds a proximity graph filtration,
    computes the profile over :math:`\\mathbb F_2`, and displays the plot.
    """
    rng = np.random.default_rng(0)
    points = rng.random((60, 2))

    filtration = proximity_graph_filtration(
        points,
        use_squared_distances=False,
        distance_tolerance=0.0,
        max_steps=80,
    )

    p = 2

    k0_dims = np.empty(filtration.n_steps, dtype=int)
    k1_dims = np.empty(filtration.n_steps, dtype=int)

    for step in range(filtration.n_steps):
        adjacency = filtration.adjacency_matrix(step, include_self_loops=False, dtype=np.int8)
        kt = k_theory_cuntz_krieger_Fp(adjacency, p=p)
        k0_dims[step] = kt.k0_dim
        k1_dims[step] = kt.k1_dim

    profile = KTheoryProfile(
        thresholds=filtration.thresholds,
        k0_dims=k0_dims,
        k1_dims=k1_dims,
        p=p,
    )

    plot_k_theory_profile(profile, title="Homolipop graph C*-K-theory profile over F_2")
    plt.show()


if __name__ == "__main__":
    main()